In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import sys
import tensorflow as tf
from tensorflow import keras
import os

sys.path.insert(0, '../../src')

from utils import df_to_xarray,read_xarray, custom_rmse

sys.path.insert(0, '../../src/preprocess')
from data_preprocess import preprocess_image_reduced,preprocess_images_nfp, inverse_scale_frame
from data_preprocess import preprocess_images, inverse_scale_image, preprocess_image_reduced

/burg/glab/users/sk4973/venv2/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(
2022-06-21 09:48:43.166697: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
dist_map = pd.read_csv("../../src/dist_map.csv",header=None).to_numpy()
dist_map = np.roll(np.fliplr(dist_map),180)
dist_map = np.repeat(dist_map[np.newaxis, :, : ], 421, axis=0)


dir_name = "../../data/data1"
val_dir_name = "../../data/data2"

chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2 = read_xarray(dir_name)



/burg/glab/users/sk4973/venv2/lib/python3.8/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'cfgrib' loading failed:
ecCodes library not found using ['eccodes', 'libeccodes.so', 'libeccodes']
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [3]:

chl_images = preprocess_image_reduced(chl.Chl.data)
sss_images = preprocess_image_reduced(sss.SSS.data)
sst_images = preprocess_image_reduced(sst.SST.data)
xco2_images = preprocess_image_reduced(xco2.XCO2.data,xco2=True)
y1 = preprocess_image_reduced(pco2.pCO2.data)
dist_map = preprocess_image_reduced(dist_map)
    

X1 = np.dstack((chl_images, dist_map, sss_images, sst_images, xco2_images))
X1 = X1.reshape((421,180,360,5),order='F')

In [4]:
chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2 = read_xarray(val_dir_name, num = "035")

chl_images = preprocess_image_reduced(chl.Chl.data)
sss_images = preprocess_image_reduced(sss.SSS.data)
sst_images = preprocess_image_reduced(sst.SST.data)
xco2_images = preprocess_image_reduced(xco2.XCO2.data,xco2=True)
y2 = preprocess_image_reduced(pco2.pCO2.data)
dist_map = preprocess_image_reduced(dist_map)
    

X2 = np.dstack((chl_images, dist_map, sss_images, sst_images, xco2_images))
X2 = X2.reshape((421,180,360,5),order='F')

In [5]:
import sklearn
from sklearn.model_selection import train_test_split

X=np.concatenate((X1, X2), axis = 0)
y=np.concatenate((y1,y2),axis = 0)


train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.3)

In [6]:
train_X_index=np.lib.stride_tricks.sliding_window_view(range(train_X.shape[0]),3) 
val_X_index=np.lib.stride_tricks.sliding_window_view(range(val_X.shape[0]),3) 

train_y=np.expand_dims(train_y[train_X_index][1:],axis=4)
train_X=train_X[train_X_index][:-1]

val_y=np.expand_dims(val_y[val_X_index][1:],axis=4)
val_X=val_X[val_X_index][:-1]

In [7]:
print("train set shape")
print(train_X.shape, train_y.shape)

print("val set shape")
print(val_X.shape, val_y.shape)

INPUT_SHAPE = train_X[0].shape



train set shape
(586, 3, 180, 360, 5) (586, 3, 180, 360, 1)
val set shape
(250, 3, 180, 360, 5) (250, 3, 180, 360, 1)


In [9]:
#tf.keras.backend.clear_session()
from functools import partial


DefaultConvLSTM2D = partial(keras.layers.ConvLSTM2D,
                        filters=32, kernel_size=(5, 5),
                        padding="same",return_sequences=True,
                        activation="elu",)



model = keras.models.Sequential([
    DefaultConvLSTM2D(input_shape=INPUT_SHAPE),
    keras.layers.BatchNormalization(),
    DefaultConvLSTM2D(kernel_size=(3,3)),
    keras.layers.BatchNormalization(),
    DefaultConvLSTM2D(kernel_size=(1,1)),
    keras.layers.Conv3D(filters = 1, kernel_size=(3,3,3),activation="elu", padding="same")
    
])


model.compile(
    loss=custom_rmse, optimizer=keras.optimizers.Adam(learning_rate=0.0005),
)

2022-06-21 09:49:35.237780: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-21 09:49:35.239077: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.62GHz coreCount: 72 deviceMemorySize: 44.49GiB deviceMemoryBandwidth: 581.23GiB/s
2022-06-21 09:49:35.239617: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-06-21 09:49:35.239649: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-06-21 09:49:35.239658: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-06-21 09:49:35.239667: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: all CUDA-capable devices are busy or unavailable

In [ ]:
model_path="../../models/ConvLSTM_wo_pco2.h5"

early_stopings = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=1, mode='min')
checkpoint =  tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
callbacks=[early_stopings,checkpoint]

epochs = 50
batch_size = 8

# Fit the model to the training data.
hist = model.fit(
    train_X,
    train_y,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(val_X,val_y),
    callbacks=callbacks,
)

In [10]:
X_index=np.lib.stride_tricks.sliding_window_view(range(421),3) 
#Xc1 = Xc1[X_index][:-1]
y1=np.expand_dims(y1[X_index][1:],axis=4)

best_model = tf.keras.models.load_model('../../models/ConvLSTM_wo_pco2.h5', custom_objects={'custom_rmse':custom_rmse})
pred1=best_model.predict(X1,verbose=1)
pred1[y1==0]=0.0


chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2t2 = read_xarray(dir_name)

y_true,y_pred = inverse_scale_frame(pred1,pco2t2.pCO2.data)

y_pred_socat = np.copy(y_pred)
y_true_socat=np.expand_dims(pco2t2.pCO2_socat.data[X_index][1:],axis=4)
y_true_socat = np.nan_to_num(y_true_socat)
y_pred_socat[y_true_socat==0]=0.0

2022-06-21 09:49:36.612254: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-21 09:49:36.613560: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.62GHz coreCount: 72 deviceMemorySize: 44.49GiB deviceMemoryBandwidth: 581.23GiB/s
2022-06-21 09:49:36.613613: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-06-21 09:49:36.613635: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-06-21 09:49:36.613644: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-06-21 09:49:36.613653: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: all CUDA-capable devices are busy or unavailable

In [ ]:
print("Full RMSE score:")
a=custom_rmse(y_pred[:,:2],y_true[:,:2])
print(a.numpy())

print("SOCAT RMSE score:")
b=custom_rmse(y_pred_socat[:,:2],y_true_socat[:,:2])
print(b.numpy())